# plotting with plotly

import psycopg2 # used to communicate and drive postgres
from sqlalchemy import create_engine # 


In [10]:
import plotly.graph_objs as go
import plotly.offline as offline

offline.init_notebook_mode(connected=True)
import numpy as np

x = np.random.randn(2000)
y = np.random.randn(2000)
offline.iplot([go.Histogram2dContour(x=x, y=y, contours=dict(coloring='heatmap')),
       go.Scatter(x=x, y=y, mode='markers', marker=dict(color='white', size=3, opacity=0.3))], show_link=False)

In [12]:
import pandas as pd
import requests # used for api calls
import json # parse json body
from pandas.io.json import json_normalize
import psycopg2 # used to communicate and drive postgres
from sqlalchemy import create_engine # 


/Users/jakekirsch/.pyenv/versions/jupyter3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



# strava api

In [30]:
import config # create a configuration file to hide your API keys

In [31]:
# this refreshes your access code - previously created an access code that had greater permissions
refresh = {'client_id':config.client_id,
          'client_secret':config.client_secret,
          'grant_type':'refresh_token',
          'refresh_token':config.refresh_token}
response = requests.post('https://www.strava.com/oauth/token', params=refresh)

In [32]:
access_token = response.json()['access_token']

In [26]:
# confirm connection - hide this for github
header = {'Authorization': 'Bearer {}'.format(access_token)}

In [27]:
athlete = requests.get('https://www.strava.com/api/v3/athlete', headers=header)

In [28]:
athletedf = json_normalize(athlete.json())

In [29]:
athlete.json()

{'athlete_type': 1,
 'badge_type_id': 0,
 'bikes': [],
 'city': None,
 'clubs': [{'admin': False,
   'city': 'San Francisco',
   'country': 'United States',
   'cover_photo': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/15879/4453767/6/large.jpg',
   'cover_photo_small': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/15879/4453767/6/small.jpg',
   'featured': False,
   'id': 15879,
   'member_count': 0,
   'membership': 'member',
   'name': 'San Francisco Running Company',
   'owner': False,
   'private': False,
   'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/15879/314178/1/large.jpg',
   'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/15879/314178/1/medium.jpg',
   'resource_state': 2,
   'sport_type': 'running',
   'state': 'California',
   'url': 'san-francisco-running-company',
   'verified': True}],
 'country': None,
 'created_at': '2015-04-11T15:31:28Z',
 'date_preference': '%m/%d/%Y',
 'firstname': 'Jake',
 'follower': None

In [19]:
# try to update weight
d = {"weight":"180"}
resp = requests.put('https://www.strava.com/api/v3/athlete', headers=header, data=d)

In [20]:
resp.json()

{'athlete_type': 1,
 'badge_type_id': 0,
 'bikes': [],
 'city': None,
 'clubs': [{'admin': False,
   'city': 'San Francisco',
   'country': 'United States',
   'cover_photo': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/15879/4453767/6/large.jpg',
   'cover_photo_small': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/15879/4453767/6/small.jpg',
   'featured': False,
   'id': 15879,
   'member_count': 0,
   'membership': 'member',
   'name': 'San Francisco Running Company',
   'owner': False,
   'private': False,
   'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/15879/314178/1/large.jpg',
   'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/15879/314178/1/medium.jpg',
   'resource_state': 2,
   'sport_type': 'running',
   'state': 'California',
   'url': 'san-francisco-running-company',
   'verified': True}],
 'country': None,
 'created_at': '2015-04-11T15:31:28Z',
 'date_preference': '%m/%d/%Y',
 'firstname': 'Jake',
 'follower': None

In [ ]:
param = {"page":1,
        "per_page":35}
activities = requests.get('https://www.strava.com/api/v3/athlete/activities', headers = header, params=param)

In [ ]:
activities.json()

In [ ]:
activitiesdf = json_normalize(activities.json(), sep="_")

In [ ]:
activitiesdf.head()

In [ ]:
# create connection and cursor to connect
def execute(q):
    with psycopg2.connect('dbname=jakekirsch user=jakekirsch') as conn:
        cur = conn.cursor()
        # we'll define a table and drop every time during testing
        cur.execute(q)


In [ ]:
drop_create_table = """
DROP TABLE IF EXISTS activities;

CREATE TABLE activities (
achievement_count int,
athlete_id text,
athlete_resource_state text,
athlete_count int,
average_cadence float,
average_heartrate float,
average_speed float,
average_temp float,
comment_count float,
commute boolean,
display_hide_heartrate_option boolean,
distance float,
elapsed_time float,
elev_high float,
elev_low float,
end_latlng text,
external_id text,
flagged boolean,
from_accepted_tag boolean,
gear_id text,
has_heartrate boolean,
has_kudoed boolean,
heartrate_opt_out boolean,
id int,
kudos_count int,
location_city text,
location_country text,
location_state text,
manual boolean,
map_id text,
map_resource_state text,
map_summary_polyline text,
max_heartrate float,
max_speed float,
moving_time float,
name text,
photo_count int,
pr_count int,
private boolean,
resource_state int,
start_date timestamp,
start_date_local timestamp,
start_latitude text,
start_latlng text,
start_longitude text,
timezone text,
total_elevation_gain float,
total_photo_count int,
trainer boolean,
type text,
upload_id int,
utc_offset float,
visibility text,
workout_type text
);"""


In [ ]:
execute(drop_create_table)

In [ ]:
# create this once
engine = create_engine('postgresql://jakekirsch:@localhost/jakekirsch')

In [ ]:
# procure a connection resource (needed to send SQL to database)
connection = engine.connect()
result = connection.execute("select * from activities")
connection.close()

In [ ]:
connection = engine.connect()
activitiesdf.to_sql('activities', con=connection, if_exists='replace', index=False)
result = connection.execute("select * from activities")
connection.close()
result.fetchone()

In [ ]:
with engine.connect() as conn:
    result = pd.read_sql("SELECT * FROM activities", con=conn)

In [ ]:
import app as dash_sample # user written file

In [ ]:
dash_sample.generate_table(result)